In [ ]:
import syft as sy
from tools import datasets
import time

sy.load('opacus')

In [ ]:
duet = sy.launch_duet(loopback=True)
num_of_data_owners = 2

In [ ]:
start_time = time.time()

# Get the whole data from the Loader
train_ds, test_ds = datasets.Loader.load_MedNIST(sample_size=0.035, test_size=0.1)
train_data, train_labels = train_ds.as_tensor()
test_data, test_labels = test_ds.as_tensor()

# Take this Data Owners split of the data 
# NOTE: The shuffle in load_MedNIST is seeded and therefore every call gets the same order
train_len = len(train_data)
train_split_len = int(1/num_data_owners * train_len)
test_len = len(test_data)
test_split_len = int(1/num_data_owners * test_len)

train_data, train_labels = train_data[train_split_len:2*train_split_len], train_labels[train_split_len:2*train_split_len]
test_data, test_labels = test_data[test_split_len:2*test_split_len], test_labels[test_split_len:2*test_split_len]

# Send his data to the duet.store
duet_start_time = time.time()

train_data.send(duet, pointable=True)
train_labels.send(duet, pointable=True)

test_data.send(duet, pointable=True)
test_labels.send(duet, pointable=True)

end_time = time.time()

print(f'Load + Send: {float(end_time - start_time)} seconds')
print(f'Send: {float(end_time - duet_start_time)} seconds')
print(train_len)
print(test_len


In [ ]:
duet.requests.add_handler(action="accept")